<a href="https://colab.research.google.com/github/yuancx2025/526_project_group2/blob/main/ds_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
import random

csv_path = '/content/drive/MyDrive/itineraries_sample.csv'

flight_sample = pd.read_csv(csv_path)
flight_sample.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,0d654f1c4884897aff1c2ffb968e6e40,2022-10-05,2022-11-08,ORD,SFO,KAU4AKBS,PT7H57M,1,True,False,...,1667969040||1667981100,2022-11-08T20:44:00.000-08:00||2022-11-09T00:0...,LAX||SFO,ORD||LAX,United||United,UA||UA,Boeing 757-300||Airbus A320,16560||5340,1745||339,coach||coach
1,b0c2b8e7f41fd456ca95b41e00d9ce56,2022-10-05,2022-11-08,PHL,LAX,UAUTC2BC,PT9H19M,0,True,False,...,1667910900||1667936940,2022-11-08T07:35:00.000-05:00||2022-11-08T11:4...,ATL||ONT,PHL||ATL,Delta||Delta,DL||DL,Boeing 737-900||Airbus A321,7500||17100,667||1897,coach||coach
2,6b2e2224f5e8cec5f02e4a1221d405b9,2022-10-05,2022-11-11,PHL,DEN,PI7QBEL1,PT14H5M,0,False,False,...,1668185040||1668208320||1668231120,2022-11-11T11:44:00.000-05:00||2022-11-11T18:1...,BOS||JFK||DEN,PHL||BOS||JFK,JetBlue Airways||JetBlue Airways||JetBlue Airways,B6||B6||B6,Airbus A320||Airbus A320||Airbus A320,4620||4500||16320,280||186||1632,coach||coach||coach
3,844531694368ca0ccd15d3fad4bf230d,2022-10-05,2022-11-10,DFW,SFO,TAA2JQBS,PT6H17M,0,True,False,...,1668083700||1668102120,2022-11-10T06:35:00.000-06:00||2022-11-10T09:4...,IAH||SFO,DFW||IAH,United||United,UA||UA,Boeing 737-700||Boeing 777,4200||15420,233||1641,coach||coach
4,bac03eeb6f097dd86d66d1c5438778fe,2022-10-05,2022-11-10,LAX,EWR,QAVTA0MC,PT8H39M,0,False,False,...,1668106680||1668123840,2022-11-10T13:58:00.000-05:00||2022-11-10T18:4...,ATL||EWR,ONT||ATL,Delta||Delta,DL||DL,Airbus A321||Boeing 737-800,13980||7680,1897||756,coach||coach


In [11]:
import pandas as pd

# classifying competitions
# columns used: startingAirport, destinationAirport, segmentsAirlineCode

#flights per route per airline
route_airline = (
    flight_sample
    .groupby(['startingAirport', 'destinationAirport', 'segmentsAirlineCode'])
    .size()
    .reset_index(name='count')
)

# market share within each route
route_airline['total_route'] = (
    route_airline.groupby(['startingAirport', 'destinationAirport'])['count']
    .transform('sum')
)
route_airline['share'] = route_airline['count'] / route_airline['total_route']

# Herfindahl–Hirschman Index (HHI)
hhi = (
    route_airline
    .groupby(['startingAirport', 'destinationAirport'])['share']
    .apply(lambda s: (s**2).sum())
    .reset_index(name='HHI')
)

# classifying market type
def classify(hhi_val):
    if hhi_val > 0.25:
        return 'Monopoly'
    elif hhi_val > 0.15:
        return 'Oligopoly'
    else:
        return 'Competitive'

hhi['MarketType'] = hhi['HHI'].apply(classify)

flight_sample = flight_sample.merge(hhi, on=['startingAirport', 'destinationAirport'], how='left')

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

print(hhi.head())


  startingAirport destinationAirport       HHI MarketType
0             ATL                BOS  0.157388  Oligopoly
1             ATL                CLT  0.249657  Oligopoly
2             ATL                DEN  0.167441  Oligopoly
3             ATL                DFW  0.163737  Oligopoly
4             ATL                DTW  0.155175  Oligopoly


In [12]:
# distribution of market types
market_counts = hhi['MarketType'].value_counts(normalize=True) * 100

print(market_counts.round(2).to_string())

MarketType
Oligopoly      67.52
Monopoly       17.09
Competitive    15.38
